# Notebook to demonstrate Zero shot and Few shot Learning

In [14]:
import pandas as pd 
%pip install langchain_groq
from langchain_groq.chat_models import ChatGroq
import pandas as pd
import numpy as np
from itertools import product
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score

Note: you may need to restart the kernel to use updated packages.


In [113]:
# Groq API and Models 
Groq_Token = open('..\\groqapi.txt', 'r').read()  # Do not share this key with anyone
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

**NOTE : DO NOT SHARE THE API KEY WITH ANYONE. DO NOT COMMIT THE API KEY TO GITHUB.**

Always do a sanity check before committing the code to github. If the key is found in the code, you will be penalized with a 0.5 marks deduction.

# Zero Shot 

In [114]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment label: Neutral

Explanation: The sentence expresses mixed sentiments. The words "amazing" and "happy" convey a positive sentiment, indicating satisfaction with the product quality and customer service. However, the phrase "delivery was delayed" expresses a negative sentiment, indicating dissatisfaction with the delivery experience. Overall, the positive and negative sentiments balance each other out, resulting in a neutral sentiment label.


# Few Shot

In [4]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Here are few examples:
1. Sentence: 'The customer service was excellent, and I received my order quickly.'
Sentiment: Positive

2. Sentence: 'The food was bland and the service was slow.'
Sentiment: Negative

3. Sentence: 'The product is okay, but it's not worth the price.'
Sentiment: Neutral

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment: Positive

Explanation: Although the sentence mentions a negative aspect ("the delivery was delayed"), the positive sentiments ("The product quality is amazing" and "I am happy with the customer service") outweigh the negative one, resulting in an overall positive sentiment. The use of the word "amazing" and "happy" also indicates a strong positive emotion, which contributes to the positive sentiment classification.


In [11]:
f = open("C:\\Users\\Rajeev Wankar\\Desktop\\Aarsh IITGN\\ES335 Machine Learning\\main\\ES335-MLAssignment1\\HAR\\UCI HAR Dataset\\features.txt", 'r').read().split('\n')

['1 tBodyAcc-mean()-X', '2 tBodyAcc-mean()-Y', '3 tBodyAcc-mean()-Z', '4 tBodyAcc-std()-X', '5 tBodyAcc-std()-Y', '6 tBodyAcc-std()-Z', '7 tBodyAcc-mad()-X', '8 tBodyAcc-mad()-Y', '9 tBodyAcc-mad()-Z', '10 tBodyAcc-max()-X', '11 tBodyAcc-max()-Y', '12 tBodyAcc-max()-Z', '13 tBodyAcc-min()-X', '14 tBodyAcc-min()-Y', '15 tBodyAcc-min()-Z', '16 tBodyAcc-sma()', '17 tBodyAcc-energy()-X', '18 tBodyAcc-energy()-Y', '19 tBodyAcc-energy()-Z', '20 tBodyAcc-iqr()-X', '21 tBodyAcc-iqr()-Y', '22 tBodyAcc-iqr()-Z', '23 tBodyAcc-entropy()-X', '24 tBodyAcc-entropy()-Y', '25 tBodyAcc-entropy()-Z', '26 tBodyAcc-arCoeff()-X,1', '27 tBodyAcc-arCoeff()-X,2', '28 tBodyAcc-arCoeff()-X,3', '29 tBodyAcc-arCoeff()-X,4', '30 tBodyAcc-arCoeff()-Y,1', '31 tBodyAcc-arCoeff()-Y,2', '32 tBodyAcc-arCoeff()-Y,3', '33 tBodyAcc-arCoeff()-Y,4', '34 tBodyAcc-arCoeff()-Z,1', '35 tBodyAcc-arCoeff()-Z,2', '36 tBodyAcc-arCoeff()-Z,3', '37 tBodyAcc-arCoeff()-Z,4', '38 tBodyAcc-correlation()-X,Y', '39 tBodyAcc-correlation()-X,Z

In [12]:
feature_labels = pd.read_csv(r'../HAR/UCI HAR Dataset/features.txt',sep = '\\s+',header=None)
dataframeX = pd.read_csv(r'../HAR/UCI HAR Dataset/train/X_train.txt',sep = '\\s+',header=None)
dataframeY = pd.read_csv(r'../HAR/UCI HAR Dataset/train/y_train.txt',sep = '\\s+',header=None)

# dataframeX.columns = feature_labels[1].tolist()
dataframeX

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.070157,-0.588433,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,0.165259,-0.390738,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,0.195034,0.025145,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,0.013865,0.063907,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339


In [13]:
correlation_matrix = dataframeX.corr()
correlation_matrix

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,1.000000,0.148061,-0.256952,0.000619,-0.021903,-0.044617,0.006290,-0.022754,-0.047558,0.044062,...,0.030681,-0.017557,-0.015613,-0.544320,0.012173,0.037444,0.028844,-0.035257,0.034371,0.028242
1,0.148061,1.000000,-0.078769,-0.045160,-0.044920,-0.049746,-0.044180,-0.045049,-0.050402,-0.038108,...,-0.022395,-0.001587,-0.004459,0.070559,-0.013541,0.017967,0.075679,-0.005309,0.001053,-0.013903
2,-0.256952,-0.078769,1.000000,-0.020217,-0.016641,-0.008410,-0.018747,-0.015203,-0.001988,-0.037197,...,-0.020481,0.020091,0.019127,0.052841,-0.039836,-0.063609,-0.034037,0.008587,-0.015288,-0.022643
3,0.000619,-0.045160,-0.020217,1.000000,0.927461,0.851668,0.998632,0.920888,0.846392,0.980844,...,-0.065987,0.148034,0.115565,-0.035011,-0.021633,0.018985,-0.024810,-0.371653,0.471065,0.394825
4,-0.021903,-0.044920,-0.016641,0.927461,1.000000,0.895510,0.922803,0.997347,0.894509,0.917366,...,-0.105621,0.206227,0.176946,-0.020379,-0.012505,-0.008507,-0.014592,-0.380531,0.523600,0.433169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,0.037444,0.017967,-0.063609,0.018985,-0.008507,-0.018429,0.019389,-0.012546,-0.023525,0.025066,...,-0.026615,0.034514,0.024553,-0.006269,0.009141,1.000000,-0.116001,-0.005853,-0.012313,-0.019903
557,0.028844,0.075679,-0.034037,-0.024810,-0.014592,-0.006471,-0.024951,-0.012341,-0.007231,-0.028871,...,0.000102,-0.017937,-0.014865,-0.020823,0.035263,-0.116001,1.000000,0.023995,-0.005869,-0.005656
558,-0.035257,-0.005309,0.008587,-0.371653,-0.380531,-0.345011,-0.368191,-0.377025,-0.347389,-0.384192,...,0.087332,-0.086006,-0.079751,0.011880,0.023246,-0.005853,0.023995,1.000000,-0.783848,-0.643655
559,0.034371,0.001053,-0.015288,0.471065,0.523600,0.476006,0.466424,0.525081,0.477607,0.480229,...,-0.100125,0.086993,0.078079,0.001540,-0.012990,-0.012313,-0.005869,-0.783848,1.000000,0.594885


In [29]:
feature_labels = pd.read_csv(r'../HAR/UCI HAR Dataset/features.txt',sep = '\\s+',header=None)
dataframeX = pd.read_csv(r'../HAR/UCI HAR Dataset/train/X_train.txt',sep = '\\s+',header=None)
correlation_matrix = dataframeX.corr()

In [39]:
def reduce_features(threshold = None):
    #feature_labels = pd.read_csv(r'../HAR/UCI HAR Dataset/features.txt',sep = '\\s+',header=None)
    #dataframeX = pd.read_csv(r'../HAR/UCI HAR Dataset/train/X_train.txt',sep = '\\s+',header=None)
    #dataframeY = pd.read_csv(r'../HAR/UCI HAR Dataset/train/y_train.txt',sep = '\\s+',header=None)
    #test_values_y = pd.read_csv(r'../HAR/UCI HAR Dataset/test/y_test.txt',sep = "\\s+",header=None)
    #test_data_X = pd.read_csv(r'../HAR/UCI HAR Dataset/test/X_test.txt',sep = "\\s+",header=None)
    #correlation_matrix = dataframeX.corr()
    removals = set()
    pairs = product(range(len(correlation_matrix.columns)), repeat=2)

    for i, j in pairs:
        if i > j:
            correlation_value = correlation_matrix.iloc[i, j]
            if abs(correlation_value) > threshold and correlation_value != 1:
                if i not in removals and j not in removals:
                    removals.add(i)
    #print(removals)
    selected_features = [col for col in dataframeX.columns if col not in removals]
    return selected_features
    
    # clf = tree.DecisionTreeClassifier(max_depth = depth)
    # clf = clf.fit(filtered_dfX, dataframeY)
    # tree.plot_tree(clf)
    
    # filtered_testx =test_data_X[selected_features]
    
    # y_pred = clf.predict(filtered_testx)
    # y_pred = np.array(y_pred)
    # y_test = test_values_y[0].to_numpy()
    # print(filtered_testx.shape)
    # accuracy = accuracy_score(y_test, y_pred)
    # print(f"Accuracy: {accuracy}")
    # precision = precision_score(y_test, y_pred,average='weighted')
    # print(f"Precision: {precision}")
    # recall = recall_score(y_test, y_pred, average='weighted')
    # print(f"Recall: {recall}")
    # cm = confusion_matrix(y_test, y_pred)
    # print("Confusion Matrix:")
    # print(cm)

    

In [43]:
frs = reduce_features(threshold=0.9)

In [107]:
features = np.array(feature_labels[1].iloc[frs])

def example(n):
    return dataframeX[frs].iloc[n, :].to_numpy()

activity_dic = {1: "WALKING", 2: "WALKING_UPSTAIRS", 3: "WALKING_DOWNSTAIRS", 4: "SITTING", 5: "STANDING", 6: "LAYING"}
def label(n):
    #print(dataframeY.iloc[n, 0])
    return (activity_dic[dataframeY.iloc[n, 0]])
    
label(10)
print(label(100))
print(label(234))
print(label(134))
print(label(135))

WALKING
SITTING
WALKING_DOWNSTAIRS
WALKING_DOWNSTAIRS


In [94]:
label(135)

WALKING_DOWNSTAIRS


In [76]:
print(len(x_example), len(features))

233 233


In [77]:
dataframeY.iloc[5314]

0    4
Name: 5314, dtype: int64

In [78]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."
# System Prompts 
query = f"""
* You are a Human Activity Recognition model.
* Your task is to analyze the data given in the features provided and guess the activity being performed by the person.
* The activities are:
1 WALKING
2 WALKING_UPSTAIRS
3 WALKING_DOWNSTAIRS
4 SITTING
5 STANDING
6 LAYING
* Provide the activity label and, if necessary, a brief explanation of your reasoning.
Feature Names: {features}
Feature Data: {x_example}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Based on the feature data provided, I predict that the activity being performed by the person is **WALKING**.

My reasoning is as follows:

* The values of `tBodyAcc-mean()-X`, `tBodyAcc-mean()-Y`, and `tBodyAcc-mean()-Z` are relatively small, indicating that the body acceleration is not too high in any direction, which is consistent with walking.
* The values of `tBodyAcc-std()-X`, `tBodyAcc-std()-Y`, and `tBodyAcc-std()-Z` are relatively low, indicating that the body acceleration is not too variable, which is also consistent with walking.
* The values of `tBodyGyro-mean()-X`, `tBodyGyro-mean()-Y`, and `tBodyGyro-mean()-Z` are relatively small, indicating that the body gyroscope is not too high in any direction, which is consistent with walking.
* The values of `fBodyAcc-bandsEnergy()-1,8`, `fBodyAcc-bandsEnergy()-9,16`, and `fBodyAcc-bandsEnergy()-17,24` are relatively high, indicating that the body acceleration has a significant energy component in the frequency range of 1-24 Hz, wh

In [105]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."
# System Prompts 
query = f"""
* You are a Human Activity Recognition model.
* Your task is to analyze the data given in the features provided and guess the activity being performed by the person.
* The activities are:
1 WALKING
2 WALKING_UPSTAIRS
3 WALKING_DOWNSTAIRS
4 SITTING
5 STANDING
6 LAYING
* Provide the activity label and, if necessary, a brief explanation of your reasoning.
Feature Names: {features}
Here are a few examples for your easy understanding:
1. Feature Data: {example(234)}
   Predicted Label: {label(234)}
2. Feature Data: {example(134)}
   Predicted Label: {label(134)}

Now, predict the activity for this:
Feature Data: {example(135)}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Based on the provided feature data, I predict the activity label as: WALKING_UPSTAIRS
